# MNIST Dataset Load

In [14]:
import os
import struct
import numpy as np

"""
Loosely inspired by http://abel.ee.ucla.edu/cvxopt/_downloads/mnist.py
which is GPL licensed.
"""

def read(dataset = "training", path = "."):
    """
    Python function for importing the MNIST data set.  It returns an iterator
    of 2-tuples with the first element being the label and the second element
    being a numpy.uint8 2D array of pixel data for the given image.
    """

    if dataset is "testing":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte')
    elif dataset is "training":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
    else:
        raise Exception("dataset must be 'testing' or 'training'")

    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)

    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)

    return img, lbl

In [15]:
# MNIST data directory path
path = 'MNIST/'

In [16]:
X_train, y_train = read("training", path)
X_test, y_test = read("testing", path)

In [17]:
# reshape matrix to vector
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

## Data Scaling - Normalization

In [28]:
from sklearn.preprocessing import scale

# Normalization per image
X_train = scale(X_train, axis=1)
X_test = scale(X_test, axis=1)

# Logistic Regression 

In [33]:
random_state = 108

## 1) One vs All Classifier

In [34]:
import pandas as pd
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

# hyper parameter candidate
penalty = ['l1', 'l2']
C       = [1e-5, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]
solvers  = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
classify_problem = ['ovr', 'multinomial']

In [38]:
df = pd.DataFrame(columns=['solver', 'penalty', 'C value', 'f1 score', 'learning time'])
bookmark = 0

for solver in solvers:
    for p in penalty:
        for c in C:
            try:
                logisticRegr = LogisticRegression(penalty=p, C=c, solver = solver,
                                                  multi_class='ovr', random_state=random_state)
                ovrc = OneVsRestClassifier(logisticRegr)
                begin_time = datetime.now()
                ovrc.fit(X_train, y_train)
                end_time = datetime.now()
                
                y_pred = ovrc.predict(X_test)
                score = f1_score(y_test, y_pred, average='macro')
                df.loc[bookmark] = [solver, p, c, score, end_time - begin_time]
                print(df.loc[bookmark])
                df.to_csv("LR_ovr.csv", mode='w')

                bookmark = bookmark + 1                
            except Exception as ex:
                print('ERROR:', ex)
                break
            finally:
                print('=======================================')

df.to_csv("LR_ovr.csv", mode='w')

ERROR: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.
solver                        newton-cg
penalty                              l2
C value                           1e-05
f1 score                       0.751356
learning time    0 days 00:00:08.139838
Name: 0, dtype: object
solver                        newton-cg
penalty                              l2
C value                          0.0001
f1 score                       0.849209
learning time    0 days 00:00:10.400178
Name: 1, dtype: object
solver                        newton-cg
penalty                              l2
C value                           0.001
f1 score                       0.885608
learning time    0 days 00:00:14.043190
Name: 2, dtype: object
solver                        newton-cg
penalty                              l2
C value                            0.01
f1 score                       0.900747
learning time    0 days 00:00:17.694433
Name: 3, dtype: object
solver                      

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


solver                        liblinear
penalty                              l1
C value                           1e-05
f1 score                      0.0180412
learning time    0 days 00:00:01.390181
Name: 20, dtype: object


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


solver                        liblinear
penalty                              l1
C value                          0.0001
f1 score                      0.0180412
learning time    0 days 00:00:01.301691
Name: 21, dtype: object
solver                        liblinear
penalty                              l1
C value                           0.001
f1 score                       0.668121
learning time    0 days 00:00:03.126281
Name: 22, dtype: object
solver                        liblinear
penalty                              l1
C value                            0.01
f1 score                        0.86082
learning time    0 days 00:00:04.165766
Name: 23, dtype: object
solver                        liblinear
penalty                              l1
C value                             0.1
f1 score                       0.897861
learning time    0 days 00:00:09.071757
Name: 24, dtype: object
solver                        liblinear
penalty                              l1
C value                 

...

solver                             saga
penalty                              l1
C value                           1e-05
f1 score                      0.0202032
learning time    0 days 00:00:00.829667
Name: 50, dtype: object


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


solver                             saga
penalty                              l1
C value                          0.0001
f1 score                       0.018066
learning time    0 days 00:00:00.974061
Name: 51, dtype: object


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


solver                             saga
penalty                              l1
C value                           0.001
f1 score                       0.427223
learning time    0 days 00:00:13.845486
Name: 52, dtype: object
solver                             saga
penalty                              l1
C value                            0.01
f1 score                       0.859994
learning time    0 days 00:01:15.697763
Name: 53, dtype: object
solver                             saga
penalty                              l1
C value                             0.1
f1 score                       0.897919
learning time    0 days 00:01:20.210955
Name: 54, dtype: object
solver                             saga
penalty                              l1
C value                               1
f1 score                       0.896223
learning time    0 days 00:01:30.326411
Name: 55, dtype: object
solver                             saga
penalty                              l1
C value                 

In [43]:
df.sort_values(by='f1 score', ascending=False).head()

,solver,penalty,C value,f1 score,learning time
3,newton-cg,l2,0.01,0.900747,00:00:17.694433
13,lbfgs,l2,0.01,0.900662,00:00:03.646381
43,sag,l2,0.01,0.900186,00:00:54.906045
44,sag,l2,0.10,0.899440,00:00:58.609193
63,saga,l2,0.01,0.899319,00:01:00.089646


## 2) Multi-class Classifier